In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cude' if torch.cuda.is_available() else 'cpu'

block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-3
eval_iters = 250